In [1]:
import pandas as pd
from torch.utils.data import Dataset
import PIL
import albumentations as A
import os
from transformers import Trainer, TrainingArguments, EfficientNetImageProcessor, EfficientNetForImageClassification
from transformers import AutoImageProcessor, Swinv2ForImageClassification
import torch
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, WeightedRandomSampler

In [ ]:
model_name = "google/efficientnet-b3"
run_name = "./efficient-net-b3-random-sample"

In [3]:
device = "cuda" if  torch.cuda.is_available else "cpu"

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
preprocessor = AutoImageProcessor.from_pretrained(model_name)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
# Аугментации

train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.3),
    A.Rotate(limit=15, p=0.5),

    A.OneOf([
        A.CLAHE(clip_limit=2),
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1),
    ], p=0.3),

    A.GaussianBlur(blur_limit=(3,5), p=0.1),
    A.GaussNoise(var_limit=(5, 20), p=0.1),
])


C:\Users\Степан\AppData\Local\Temp\ipykernel_17768\1894441869.py:12: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5, 20), p=0.1),


In [8]:
class ISICDataset_train(Dataset):
    def __init__(self, image_dir, df, preprocessor, transform=None):
        self.image_dir = image_dir
        self.preprocessor = preprocessor
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __getitem__(self, idx):
        image_name = self.df.iloc[idx]['image_name'] + '.jpg'
        image_path = os.path.join(self.image_dir, image_name)

        with PIL.Image.open(image_path) as img:
            image = img.convert("RGB")

        if self.transform:
            augmented = self.transform(image=np.array(image))
            image = augmented['image']
            if isinstance(image, torch.Tensor):
                image = image.permute(1, 2, 0).cpu().numpy()

        inputs = self.preprocessor(image, return_tensors="pt")
        pixel_values = inputs["pixel_values"].squeeze(0)

        label = int(self.df.iloc[idx]['target'])
        return {"pixel_values": pixel_values, "labels": label}

    def __len__(self):
        return len(self.df)
    
    def show_image(self, idx):
        image_name = self.df.iloc[idx]['image_name'] + '.jpg'
        image_path = os.path.join(self.image_dir, image_name)

        with PIL.Image.open(image_path) as img:
            image = img.convert("RGB")

        if self.transform:
            augmented = self.transform(image=np.array(image))
            image = augmented["image"]
        else:
            image = np.array(image)

        if isinstance(image, torch.Tensor):
            image = image.permute(1, 2, 0).cpu().numpy()

        plt.imshow(image)
        plt.axis('off')
        plt.title(f"Index: {idx}, Label: {self.df.iloc[idx]['target']}")
        plt.show()

In [9]:
train_data, val_data = train_test_split(train, test_size=0.05, random_state=42, shuffle=True, stratify=train['target'])

train_dataset = ISICDataset_train("train", train_data, preprocessor, transform=train_transform)
val_dataset = ISICDataset_train("train", val_data, preprocessor)

In [10]:
# model = EfficientNetForImageClassification.from_pretrained(
#     model_name, 
#     use_safetensors=True,
#     num_labels=2,
#     ignore_mismatched_sizes=True
# )

In [ ]:
# Загрузка из чекпоинта 

model = EfficientNetForImageClassification.from_pretrained(
    "efficient-net-b3-random-sample/checkpoint-1200", 
    use_safetensors=True,
    num_labels=2,
    ignore_mismatched_sizes=True
)

In [ ]:
# # Пример заморозки backbone модели


# for param in model.efficientnet.parameters():
#     param.requires_grad = False

In [ ]:
# class MyTrainer(Trainer):
#     def get_train_dataloader(self):
#         return DataLoader(
#             self.train_dataset,
#             batch_size=self.args.train_batch_size,
#             shuffle=True,
#             num_workers=self.args.dataloader_num_workers,
#             prefetch_factor=4,          # <-- ваш параметр
#             pin_memory=True,
#             collate_fn=self.data_collator,
#         )


In [ ]:
def compute_metrics(pred):
    preds = torch.tensor(pred.predictions)
    labels = torch.tensor(pred.label_ids)
    probs = torch.softmax(preds, dim=1)
    pred_classes = torch.argmax(probs, dim=1)
    f1 = f1_score(labels, pred_classes, average="weighted")
    auc = roc_auc_score(labels.numpy(), probs[:, 1].numpy())
    precision = precision_score(labels, pred_classes, average="weighted")
    recall = recall_score(labels, pred_classes, average="weighted")

    # --- Пример КАСТОМНОЙ сложной метрики ---
    num_classes = preds.shape[1]
    ber_list = []

    for cls in range(num_classes):
        # True Positives и False Negatives для данного класса
        tp = ((pred_classes == cls) & (labels == cls)).sum().item()
        fn = ((pred_classes != cls) & (labels == cls)).sum().item()

        # Добавим защиту от деления на ноль
        if tp + fn > 0:
            fn_rate = fn / (tp + fn)
        else:
            fn_rate = 0.0

        ber_list.append(fn_rate)

    # Финальная кастомная метрика
    ber = sum(ber_list) / len(ber_list)
    return {
        "f1": f1,
        "precision": precision,
        "recall": recall,
        "auc_roc": auc,
        "balanced_error_rate": ber
    }

training_args = TrainingArguments(
    # Основные параметры
    output_dir=run_name,  # Директория для сохранения
    
    # Параметры обучения
    num_train_epochs=5,                     # Количество эпох
    per_device_train_batch_size=24,         # Размер батча для обучения
    per_device_eval_batch_size=24,          # Размер батча для валидации
    learning_rate=3e-5,                     # Learning rate
    warmup_ratio = 0.1,                     # 10% от общего числа шагов для вармапа или warmup_steps = int(0.1 * total_training_steps)
    lr_scheduler_type = 'cosine',           # Можете посмотреть на них в 
                                            # https://www.kaggle.com/code/snnclsr/learning-rate-schedulers 
                                            # соответсвующий ему будет get_cosine_schedule_with_warmup
    gradient_accumulation_steps=4,
    # Сохранение и логирование
    logging_dir='./logs',                   # Директория для логов
    logging_steps=20,                      # Частота логирования
    save_steps=300,                         # Частота сохранения
    save_total_limit=2,                     # Максимум чекпоинтов
    save_strategy='steps',                  # Стратегия сохранения
    
    # dataloader_num_workers=4,

    # Валидация
    eval_strategy='steps',
    eval_steps=300,            # Стратегия валидации
    load_best_model_at_end=True,            # Загружать лучшую модель
    metric_for_best_model='auc_roc',
    greater_is_better=True,                 # Больше значение = лучше
    # воспроизводимость
    seed=42,                                # Seed для воспроизводимости
)

In [15]:
# num_pos = len(train_dataset.df[train_dataset.df['target'] == 1])
# num_neg = len(train_dataset.df[train_dataset.df['target'] == 0])
# weight = torch.tensor([1.0, num_neg / num_pos]).to("cuda")  # [w0, w1]
# class WeightedTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
#         labels = inputs.pop("labels")
#         outputs = model(**inputs)
#         logits = outputs.get("logits")  # [B, num_labels]

#         loss_fct = CrossEntropyLoss(weight=weight)
#         loss = loss_fct(logits, labels.long())

#         return (loss, outputs) if return_outputs else loss

class_counts = train_data['target'].value_counts().to_dict()
weights = [1.0 / class_counts[label] for label in train_data['target']]

class RandomSamplerTrainer(Trainer):
    def get_train_dataloader(self):
        dataset = self.train_dataset
        
        sampler = WeightedRandomSampler(
            weights=weights,
            num_samples=len(dataset),
            replacement=True
        )
        
        return DataLoader(
            dataset,
            batch_size=self.args.train_batch_size,
            sampler=sampler,
            collate_fn=self.data_collator,
            num_workers=self.args.dataloader_num_workers,
        )

In [16]:
trainer = RandomSamplerTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [16]:
trainer.train()

Step,Training Loss,Validation Loss,F1,Precision,Recall,Auc Roc
300,0.393700,0.378831,0.878812,0.980023,0.808087,0.919999
600,0.267500,0.267805,0.925921,0.983345,0.885938,0.940502
900,0.222900,0.149879,0.957210,0.981419,0.940253,0.942726
1200,0.189500,0.161560,0.953437,0.981815,0.933615,0.946391
1500,0.172500,0.163402,0.950118,0.980856,0.928183,0.922710


KeyboardInterrupt: 

In [16]:
test['target'] = 0
test_dataset = ISICDataset_train("test", test, preprocessor)

In [17]:
predictions = trainer.predict(test_dataset)

c:\ProgramData\anaconda3\envs\DL\Lib\site-packages\sklearn\metrics\_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
c:\ProgramData\anaconda3\envs\DL\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [18]:
logits = torch.tensor(predictions.predictions)

probs = torch.softmax(logits, dim=1)[:, 1]

In [19]:
submission_df = pd.DataFrame({
    "image_name": test['image_name'],
    "target": probs
})
submission_df.to_csv("submission2.csv", index=False)

In [17]:
full_train_dataset = ISICDataset_train("train", train, preprocessor)
full_train_predictions = trainer.predict(full_train_dataset)
full_train_logits = torch.tensor(predictions.predictions)
full_train_probs = torch.softmax(logits, dim=1)[:, 1]
submission_df = pd.DataFrame({
    "image_name": train['image_name'],
    "target": full_train_probs
})
submission_df.to_csv("train_submission2.csv", index=False)

KeyboardInterrupt: 